# Generate a QC report for the preliminary data

The QC report consists of a table with the cell line, seeding density, and percentage failed single-cells

In [1]:
import pathlib
import pandas as pd
from pycytominer import annotate
import pprint

In [2]:
# path for platemap directory
platemap_dir = pathlib.Path("../0.download_data/metadata/platemaps")

# load in barcode platemap
barcode_platemap = pd.read_csv(
    pathlib.Path(f"{platemap_dir}/Barcode_platemap_pilot_data.csv")
)

# path for qc results (indices)
qc_results_dir = pathlib.Path("./qc_results")

# Path to dir with converted data from single-cell QC
converted_dir = pathlib.Path("./data/converted_profiles")

# output path for reports
output_dir = pathlib.Path("./qc_report")
output_dir.mkdir(parents=True, exist_ok=True)

# extract the plate names from the file name
plate_names = [file.stem.split("_")[0] for file in converted_dir.glob("*.parquet")]

In [3]:
# create plate info dictionary
plate_info_dictionary = {
    name: {
        "converted_path": (
            str(
                pathlib.Path(
                    list(converted_dir.rglob(f"{name}_converted.parquet"))[0]
                ).resolve(strict=True)
            )
            if list(converted_dir.rglob(f"{name}_converted.parquet"))
            else None
        ),
        "qc_results_path": (
            str(
                pathlib.Path(
                    list(qc_results_dir.rglob(f"{name}_failed_qc_indices.csv.gz"))[0]
                ).resolve(strict=True)
            )
            if list(qc_results_dir.rglob(f"{name}_failed_qc_indices.csv.gz"))
            else None
        ),
        # Find the platemap file based on barcode match and append .csv
        "platemap_path": (
            str(
                pathlib.Path(
                    list(
                        platemap_dir.rglob(
                            f"{barcode_platemap.loc[barcode_platemap['barcode'] == name, 'platemap_file'].values[0]}.csv"
                        )
                    )[0]
                ).resolve(strict=True)
            )
            if name in barcode_platemap["barcode"].values
            else None
        ),
        # Get the time_point based on the barcode match
        "time_point": (
            barcode_platemap.loc[
                barcode_platemap["barcode"] == name, "time_point"
            ].values[0]
            if name in barcode_platemap["barcode"].values
            else None
        ),
    }
    for name in plate_names
}

# Display the dictionary to verify the entries
pprint.pprint(plate_info_dictionary, indent=4)

{   'BR00143976': {   'converted_path': '/home/jenna/pediatric_cancer_atlas_profiling/3.preprocessing_features/data/converted_profiles/BR00143976_converted.parquet',
                      'platemap_path': '/home/jenna/pediatric_cancer_atlas_profiling/0.download_data/metadata/platemaps/Assay_Plate1_platemap.csv',
                      'qc_results_path': '/home/jenna/pediatric_cancer_atlas_profiling/3.preprocessing_features/qc_results/BR00143976_failed_qc_indices.csv.gz',
                      'time_point': 24},
    'BR00143977': {   'converted_path': '/home/jenna/pediatric_cancer_atlas_profiling/3.preprocessing_features/data/converted_profiles/BR00143977_converted.parquet',
                      'platemap_path': '/home/jenna/pediatric_cancer_atlas_profiling/0.download_data/metadata/platemaps/Assay_Plate2_platemap.csv',
                      'qc_results_path': '/home/jenna/pediatric_cancer_atlas_profiling/3.preprocessing_features/qc_results/BR00143977_failed_qc_indices.csv.gz',
         

In [7]:
# Set metadata columns to load in for the converted df
metadata_cols = [
        "Metadata_ImageNumber",
        "Image_Metadata_Plate",
        "Image_Metadata_Well",
        "Image_Metadata_Site",
        "Metadata_Nuclei_Location_Center_X",
        "Metadata_Nuclei_Location_Center_Y"
    ]

qc_report_list = []  # Initialize an empty list to store per-plate QC reports

# Generate QC report across plates
for plate, info in plate_info_dictionary.items():
    print(f"Generating QC report for {plate}")

    # Load in only the metadata columns from the converted dataframe and create a column called failed_qc
    converted_df = pd.read_parquet(info["converted_path"], columns=metadata_cols)
    converted_df["failed_qc"] = False  # Initialize all rows as False

    # Load in the qc_results_path and use the indices to change the rows that match to failing QC
    qc_failed_indices = pd.read_csv(info["qc_results_path"], compression='gzip')

    # Update failed_qc for rows matching the indices in qc_failed_indices
    converted_df.loc[qc_failed_indices["original_indices"], "failed_qc"] = True

    # Make sure that this worked by asserting that the number of True failed rows matches the number of failed indices
    num_failed_qc = converted_df["failed_qc"].sum()
    num_qc_failed_indices = len(qc_failed_indices)

    assert num_failed_qc == num_qc_failed_indices, f"Mismatch: {num_failed_qc} != {num_qc_failed_indices}"

    # Load platemap
    platemap_df = pd.read_csv(info["platemap_path"])

    # Add cell line and seeding density metadata
    annotated_df = annotate(
        profiles=converted_df,
        platemap=platemap_df,
        join_on=["Metadata_well", "Image_Metadata_Well"],
    )

    # Add 'Metadata_time_point' column based on the plate's time_point from dict
    annotated_df["Metadata_time_point"] = info["time_point"]

    # Group by cell line and seeding density, and calculate total nuclei segmented and failed QC
    failure_stats = (
        annotated_df.groupby(["Metadata_cell_line", "Metadata_seeding_density", "Metadata_time_point"])
        .agg(
            total_nuclei_segmented=("failed_qc", "count"),
            total_failed_qc=("failed_qc", "sum"),
            percentage_failing_cells=("failed_qc", "mean")
        )
        .reset_index()
    )

    # Convert to percentage
    failure_stats["percentage_failing_cells"] *= 100

    # Append to list
    qc_report_list.append(failure_stats)

# Concatenate all reports into a single DataFrame
qc_report_df = pd.concat(qc_report_list, ignore_index=True)

# Save QC report as parquet file
qc_report_df.to_parquet(pathlib.Path(f"{output_dir}/qc_report.parquet"))

# Filter the QC report to only include rows for the A673 cell line
filter_qc_report_df = qc_report_df[qc_report_df["Metadata_cell_line"] == "G402"]

# Display filtered QC report info
print(filter_qc_report_df.shape)
filter_qc_report_df

Generating QC report for BR00143979
Generating QC report for BR00143980
Generating QC report for BR00143977
Generating QC report for BR00143981
Generating QC report for BR00143978
Generating QC report for BR00143976
(15, 6)


,Metadata_cell_line,Metadata_seeding_density,Metadata_time_point,total_nuclei_segmented,total_failed_qc,percentage_failing_cells
60,G402,1000,72,3101,30,0.967430
61,G402,2000,72,9483,118,1.244332
62,G402,4000,72,15451,390,2.524108
63,G402,8000,72,22582,1059,4.689576
64,G402,12000,72,7771,448,5.765024
188,G402,1000,48,1332,35,2.627628
189,G402,2000,48,3513,53,1.508682
190,G402,4000,48,8958,88,0.982362
191,G402,8000,48,15673,358,2.284183
192,G402,12000,48,23476,1037,4.417277
